__COMP9727 PROJECT:A STEAM GAME RECOMMENDER SYSTEM__


# Introduction
This model is a recommender system for Steam games. It is based on the dataset from Steam, which is a digital distribution platform for video games. The dataset contains 316 games and 21 million users. The recommender system is based on collaborative filtering and content based model. The system integrates the recommender module and a user interaction module.

__PIPELINE__:
1. Data preprocessing, Analysis and Visualization
2. Extracting features from the game dataset, build feature vectors for games. The item vector is a hierarchical structure. First, for the description, use text vectorization technology (such as TF-IDF or Word2Vec) to process the description text of the game to generate a dense numerical vector for each game. For Tags, we still use TF-IDF. But since the game tag describes the game more significantly, we manually increase the TF-IDF weight of the tag.

__Inporting Libraries__

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

__Data Preprocessing__

In [2]:
file_path_reviews = "new_steam_reviews.csv"
file_path_desc = "new_desc.csv"
file_path_tags = "new_tags.csv"

df_reviews = pd.read_csv(file_path_reviews)
df_count = df_reviews['author.steamid'].value_counts().to_frame().reset_index().rename(columns={'author.steamid': 'count'})
df_reviews = df_reviews.loc[df_reviews['author.steamid'].isin(df_count.query("count >= 10")['index'].values), :]
df_desc = pd.read_csv(file_path_desc)
df_tags = pd.read_csv(file_path_tags)

In [3]:
print(df_reviews.isna().any().any())
print(df_desc.isna().any().any())

True
False


In [4]:
# print(df_desc)
# print(df_tags)
print(df_reviews['author.steamid'])

10          76561198184596858
30          76561198063207357
39          76561198177937343
61          76561198346767187
86          76561198938824713
                  ...        
17223453    76561198202928831
17223456    76561198030410472
17223461    76561198005772688
17223462    76561197961194023
17223465    76561198013409616
Name: author.steamid, Length: 1001740, dtype: int64


__Clean the data__

In [5]:
import ast
import re
def tags_preprocess(tag):
    tag = ast.literal_eval(tag)
    tag = [x.lower() for x in tag]
    tag = [x.replace(' ', '_') for x in tag]
    tag = [x.replace('-', '_') for x in tag]
    tag = ' '.join(tag)
    return tag
def desc_preprocess(desc):
    desc = '' if pd.isna(desc) else str(desc)
    desc = desc.lower()
    desc = re.sub(r'[^\w\s]', '', desc)  # Remove punctuation
    words = desc.split()  # Tokenize by splitting on whitespace
    return ' '.join(words)

In [6]:
# df_desc = df_desc.dropna(subset=['short_description'])
df_tags['tags'] = df_tags['tags'].apply(lambda x: tags_preprocess(x))
df_desc['short_description'] = df_desc['short_description'].apply(desc_preprocess)

__Extracting Features from the description dataset and the tags dataset__

In [7]:
print(df_desc.loc[161])

id                                                              578080
short_description    pubg battlegrounds is a battle royale shooter ...
Name: 161, dtype: object


In [8]:
import ast
tfidf_vectorizer_desc = TfidfVectorizer(stop_words='english')
tfidf_matrix_desc = tfidf_vectorizer_desc.fit_transform(df_desc['short_description'].tolist())
print(tfidf_matrix_desc)

  (0, 2342)	0.34266551688316627
  (0, 2557)	0.20954880668834158
  (0, 3875)	0.31922482778360933
  (0, 1695)	0.5793861041476892
  (0, 3115)	0.2557120159913009
  (0, 2345)	0.5793861041476892
  (1, 1561)	0.19705558180487137
  (1, 2403)	0.19705558180487137
  (1, 2657)	0.15540662450806467
  (1, 1975)	0.1835756183978875
  (1, 3375)	0.19705558180487137
  (1, 2451)	0.08627417555433464
  (1, 1644)	0.15096728250238467
  (1, 281)	0.15096728250238467
  (1, 3442)	0.1835756183978875
  (1, 1394)	0.12590762604844555
  (1, 3619)	0.29410301067932054
  (1, 139)	0.15540662450806467
  (1, 106)	0.20572695279440442
  (1, 3609)	0.19705558180487137
  (1, 341)	0.16053146874664415
  (1, 824)	0.19705558180487137
  (1, 450)	0.19705558180487137
  (1, 3377)	0.5220342964608841
  (1, 823)	0.39411116360974274
  :	:
  (274, 1178)	0.14277292503568564
  (274, 3906)	0.14277292503568564
  (274, 2854)	0.14277292503568564
  (274, 1886)	0.14277292503568564
  (274, 3904)	0.14277292503568564
  (274, 198)	0.2855458500713713
  (27

Create user class. The user class takes user model as input. A user could be an existed user in our reviews data set either a new user, needing cold start which will be implemented later.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import pandas as pd

class User_CB():
    def __init__(self, user_mod, user_id = None):
        self.user_mod = user_mod
        self.user_id = user_id
        self.profile = []  # This array contains game ids.
        self.feature_arr = None  # Feature array (TF-IDF) depends on the profile array.
        self.recommendations = []
        self.liked_tags = Counter()
        self.limit = 100
        self.total_sum = 0

    def hot_start(self):
        df_user_reviews = df_reviews[(df_reviews['author.steamid'] == self.user_id)]
        for app_id in df_user_reviews['app_id'].values:
            self.like_game(app_id)


    def cold_start(self, liked_games):
        for id in liked_games:
            self.like_game(id)

    def update_profile(self, app_id):
        self.profile.append(app_id)
        # Get tags for the new game and update tag counts
        new_game_tags = df_tags[df_tags['app_id'] == app_id]['tags'].iloc[0].split()
        self.liked_tags.update(new_game_tags)
        self.generate_new_profile_vector()

    def generate_new_profile_vector(self):
        desc = df_desc.loc[df_desc['id'].isin(self.profile)]
        self.feature_arr = tfidf_vectorizer_desc.transform(desc['short_description'])


    def like_game(self, app_id):
        # if not app_id in self.profile:
        self.update_profile(app_id)

    def run_model(self):
        cosine_similarities = cosine_similarity(self.feature_arr, tfidf_matrix_desc)
        adjusted_similarity_scores = []

        for game_idx, game_id in enumerate(df_desc['id']):
            #if game_id not in self.profile:  # Skip games already liked
            custom_weight = self.calculate_custom_weight(game_id)
            adjusted_score = (cosine_similarities[0][game_idx]+1) * custom_weight
            adjusted_similarity_scores.append((game_id, adjusted_score))

        # Create a DataFrame from the adjusted scores
        similarity_scores_df = pd.DataFrame(adjusted_similarity_scores, columns=['game_id', 'similarity_score'])
        sorted_similarity_scores_df = similarity_scores_df.sort_values(by='similarity_score', ascending=False)
        top_recommendations_indices = sorted_similarity_scores_df[~sorted_similarity_scores_df['game_id'].isin(self.profile)].index


        final_recommendations = pd.DataFrame({
            'app_id': df_desc.loc[top_recommendations_indices]['id']
        })

        # Store the recommendations
        self.recommendations = final_recommendations
        return self.recommendations['app_id'].values.tolist()

    def calculate_custom_weight(self, game_id):
        game_tags_list = df_tags[df_tags['app_id'] == game_id]['tags'].iloc[0].split()
        weight_sum = 0
        total_sum = sum(dict(self.liked_tags).values())
        for tag in game_tags_list:
            if tag in dict(self.liked_tags):
                weight_sum += self.liked_tags[tag]
        return (weight_sum/total_sum) if total_sum > 0 else 0




In [10]:
df_games= df_reviews.drop_duplicates(subset='app_id')
print(df_games['app_name'])

10                            Half-Life
57370            Counter-Strike: Source
175417         Half-Life 2: Episode Two
198924                         Portal 2
431235                        X Rebirth
                       ...             
17152364       Styx: Shards of Darkness
17155572        DRAGON QUEST HEROES™ II
17156171    Steel Division: Normandy 44
17161378                         DiRT 4
17172395                Half-Life: Alyx
Name: app_name, Length: 275, dtype: object


The core idea of collaborative filtering recommendation system is to construct the user-items matrix through the user's comments on the game, calculate the similarity between users to predict the user's rating of the game, and introduce the weight to ensure the credibility of similar users

**Pipeline:**

1. Data sampling
2. Construct the user-item matrix
3. Construct the similarity function
4. Construct the rating prediction function
5. Implement a complete recommendation process




1. Data sampling

The original dataset presents a formidable challenge due to its overwhelming number of users, which poses a significant hurdle in constructing the User-Item matrix without overtaxing the available RAM. To tackle this issue effectively, the implementation of a thoughtful data sampling strategy becomes imperative. Our primary goal is to strategically curtail the user count, thereby enhancing the efficiency of the data processing pipeline.

Upon a detailed exploration of the dataset, a notable observation emerged: approximately 90% of the 12,000,000 users contributed more than 10 comments. Leveraging this insight, we refined our selection criteria, ultimately narrowing down the user pool to approximately 120,000 active users. This deliberate curation ensures that our subsequent analyses are not only more resource-efficient but also tailored to the subset of users who exhibit significant engagement, enhancing the relevance and depth of our findings.

2. Construct the user-item matrix

The construction of the user-item matrix was based on sampled dataset in the last step, comprising 120,000 distinct users and 315 unique games. Despite our efforts to filter out inactive users, the resulting matrix remains highly sparse.


3. Construct the similarity function

In our initial approach to the similarity function, we employed cosine similarity, a widely discussed method covered in our lectures, to compute the similarity between users. However, given that our user-items matrix comprises **120,000** users and **315** games, with many users reviewing only approximate 15 games, the 0-1 matrix is highly sparse. Cosine similarity proved ineffective for dealing with vectors that contain numerous empty values.

To address this challenge, we explored alternative similarity metrics, including Pearson Correlation Coefficient, Spearman's Rank Correlation Coefficient, and Kendall's Tau. Despite these efforts, we encountered less than perfect results. Ultimately, we opted to enhance the dataset by smoothing it, replacing 0s with 1s and empty values with 0s.

Our rationale for this transformation lies in our consideration that users who review a game, regardless of recommending it or not, exhibit an interest in the genre(s) of the game. Since users invest time playing a particular game, we assume they are inherently interested in its genre. Furthermore, if a user does not recommend a game, it suggests that the game itself lacks sufficient allure. Therefore, we chose to convert 0s to 1s, reflecting the user's implicit interest in the genre associated with the game.

4. Construct the rating prediction function

The rating prediction function was implemented with the equation taught through lecture.

$\hat{r}_{ui} = \frac{\sum_{j=1}^{M}  \text{cred}(v_j, i) \cdot \text{sim}(u, v_j) \cdot r_{v_j, i}}{\sum_{j=1}^{M} \text{cred}(v_j, i) \cdot \text{sim}(u, v_j)}$

$\text{sim}(u, v_j)$ refers the similarity between $u$ and $v_j$

$\text{cred}(v_j, i)$ refers to how trustworthy the $v_j$'s rating on $i$

$\text{cred}(v_j, i)$ is implemented **......**

After obtaining predicted ratings for all games, we will select the top N games to be recommended.

In [11]:
import bisect
import numpy as np


class User_CF():
    def __init__(self, user_id):
        self.user_id = user_id
        self.recommendations = []
        # Generate users-items matrix
        if user_id == 1:
          self.cold_start()
        else:
          self.u_a_df = df_reviews.pivot_table(index='author.steamid', columns='app_id', values='recommended')
          self.reviewed_games_ids = set(self.u_a_df.loc[user_id, self.u_a_df.columns[~self.u_a_df.loc[user_id].isna()]].index.tolist())

        # dynamic
        self.buffer = []

        # Intialize interest and rating matrix
        self.interest_matrix = self.u_a_df.replace(0, 1)
        self.interest_matrix = self.interest_matrix.fillna(0)
        self.rating_matrix = self.u_a_df.replace(0, -1)
        self.rating_matrix = self.rating_matrix.fillna(0)

    def cold_start(self):
        self.u_a_df = df_reviews.pivot_table(index='author.steamid', columns='app_id', values='recommended')
        self.u_a_df.loc[1] = [np.nan] * len(self.u_a_df.columns)
        self.reviewed_games_ids = set()

    def similarity(self):
        interest_matrix = self.interest_matrix
        sim_matrix = self.cosine(interest_matrix)
        sorted_df = sim_matrix.sort_values(by='Similarity', axis=1, ascending=False)
        return sorted_df

    def cosine(self, matrix):
        uid = self.user_id
        target_user_ratings = matrix.loc[uid].to_frame().T
        similarity_scores = cosine_similarity(target_user_ratings, matrix)
        similarity_df = pd.DataFrame(similarity_scores, columns=matrix.index, index=['Similarity'])
        return similarity_df

    def comment(self, aid):
        if aid in self.reviewed_games_ids:
          return
        self.buffer.append(aid)

    def update_profile(self):
        uid = self.user_id
        for i in self.buffer:
            self.reviewed_games_ids.add(i)
        for aid in self.buffer:
          self.interest_matrix.loc[uid][aid] = 1
          self.rating_matrix.loc[uid][aid] = 1
        self.buffer = []

    def run_model_cf(self):

        self.update_profile()

        # Predict rating by 5 similar users
        similar_users = self.similarity()
        ratings = []
        app_list = self.u_a_df.columns.tolist()
        for aid in app_list:
          if aid in self.reviewed_games_ids:
            continue
          count = 0
          up = 0
          down = 0
          user_list = similar_users.columns.tolist()[1:]
          for uid in user_list:
            if count == 5:
              break

            rating = self.rating_matrix.loc[uid][aid]

            if rating == 0 or uid == self.user_id:
              continue

            count += 1
            up += similar_users.iloc[0][uid] * rating * self.credibility(uid, aid)
            down += similar_users.iloc[0][uid] * self.credibility(uid, aid)

          final_rating = up / down
          ratings.append((final_rating, aid))

        ratings.sort()
        self.recommendations = [v for _, v in ratings]

        return self.recommendations


    def credibility(self, user_id, app_id):
        df = df_reviews

        def rank(sorted_list, value):
          # bisect_left function returns the index of the inserted value, which is the point that can be inserted while keeping the list sorted
          index = bisect.bisect_left(sorted_list, value)
          # Check if the index is in the range of the list and if the value of the list at the index is equal to the value we are looking for
          if index != len(sorted_list) and sorted_list[index] == value:
            return index + 1
          return -1

        def give_vote_score():
          vote_helpful = df.loc[(df['author.steamid'] == user_id) & (df['app_id'] == app_id), 'votes_helpful'].values[0]
          vote_funny = df.loc[(df['author.steamid'] == user_id) & (df['app_id'] == app_id), 'votes_funny'].values[0]
          if vote_helpful == 0:
            vote_score = 1
          elif 0 < vote_helpful < 10:
            vote_score = 2
          elif 10 <= vote_helpful < 50:
            vote_score = 3
          elif 50 <= vote_helpful < 100:
            vote_score = 4
          elif vote_helpful >= 100:
            vote_score = 5
          if vote_funny >= 10:
            vote_score += 1
          return vote_score

        def give_time_score():
          play_time = df.loc[(df['author.steamid'] == user_id) & (df['app_id'] == app_id), 'author.playtime_at_review'].values[0]
          if play_time < 100:
            time_score = 1
          elif 100 <= play_time < 500:
            time_score = 2
          elif 500 <= play_time < 1000:
            time_score = 3
          elif play_time >= 1000:
            time_score = 4
          elif np.isnan(play_time):
            time_score = 0.1

          return time_score

        # Base score
        Base_score = give_vote_score() + give_time_score()
        # timestamp_updated
        timestamp = df.loc[df['app_id'] == app_id, 'timestamp_updated'].sort_values().values.tolist()
          # Bigger timestamp implies newer review
        user_timestamp = df.loc[(df['app_id'] == app_id) & (df['author.steamid'] == user_id), 'timestamp_updated'].values[0]
        rank = rank(timestamp, user_timestamp)
        newest_per = rank / len(timestamp)
        score = Base_score * newest_per
        # steam_purchase
        if df.loc[(df['author.steamid'] == user_id) & (df['app_id'] == app_id), 'steam_purchase'].values[0] == False:
          score *= 0.5
        # written_during_early_access
        if df.loc[(df['author.steamid'] == user_id) & (df['app_id'] == app_id), 'written_during_early_access'].values[0] == True:
          score *= 0.5

        return round(score, 2)

In [12]:
user_cb = User_CB(0)
user_cb.cold_start([524580,512900])

__UI MODULE__

In [14]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import HTML
class SteamGameRecommenderUI:
    def __init__(self):
        self.temp_liked_games = []
        self.user_id = None
        self.current_page = 0
        self.user_cb = None
        self.user_cf = None
        self.limit = 60
        self.recommended_games_ids = []
        self.login_input = widgets.Text(description='User ID:')
        self.login_button = widgets.Button(description='Login')
        self.reg_button = widgets.Button(description='Register')
        self.next_button = widgets.Button(description='Next')
        self.next_page_button = widgets.Button(description='Next Page')
        self.previous_page_button = widgets.Button(description='Previous Page')
        self.search_input = widgets.Text(description='Search Game:')
        self.search_button = widgets.Button(description='Search')
        self.output_area = widgets.Output()
        self.refresh_button = widgets.Button(description='Refresh')
        self.refresh_button.on_click(self.on_refresh_clicked)

        self.login_button.on_click(self.on_login_clicked)
        self.reg_button.on_click(self.on_register_clicked)
        self.next_page_button.on_click(self.on_next_page_clicked)
        self.previous_page_button.on_click(self.on_previous_page_clicked)
        self.search_button.on_click(self.on_search_clicked)

    def on_register_clicked(self, b):
        self.display_registration_screen()
    def display_registration_screen(self):
        with self.output_area:
            clear_output()
            random_games = random.sample(df_games['app_name'].tolist(), 10)
            games_vbox = widgets.VBox()
            for game_name in random_games:
                game_button = widgets.Button(description=game_name, layout=widgets.Layout(width='auto'))
                game_button.on_click(lambda b, game=game_name: self.on_game_selected(b, game))
                games_vbox.children += (game_button,)

            self.next_button.disabled = True
            self.next_button.on_click(self.on_next_clicked)
            display(widgets.VBox([games_vbox, self.next_button]))

    def on_game_selected(self, b, game):
        game_id = df_games.loc[df_games['app_name'] == game]['app_id'].tolist()[0]
        self.temp_liked_games.append(game_id)
        b.description = f'{game} (Selected)'
        b.disabled = True
        b.button_style = 'success'
        self.next_button.disabled = len(self.temp_liked_games) == 0


    def on_next_clicked(self, b):
        self.user_cb = User_CB(0)
        self.user_cf = User_CF(1)
        self.user_cb.cold_start(self.temp_liked_games)
        for id in self.temp_liked_games:
            self.user_cf.comment(id)
        recommendations_cb = self.user_cb.run_model()
        recommendations_cf = self.user_cf.run_model_cf()
        self._load_games_db(recommendations_cb,recommendations_cf)
        self.reset_current_page()
        self.display_recommended_games()

    def _load_games_db(self, recommendations_cb,recommendations_cf):
        rank = []
        for app_id in recommendations_cf:
            score = 0.4* (recommendations_cb.index(app_id)+1) + 0.6 * (recommendations_cf.index(app_id)+1)
            rank.append((score, app_id))
        rank.sort()
        self.recommended_games_ids = [v for _, v in rank]


    def display(self):
        login_hbox = widgets.HBox([self.login_input, self.login_button, self.reg_button])
        display(login_hbox, self.output_area)

    def on_login_clicked(self, b):
        self.user_id = int(self.login_input.value)
        self.user_cb = User_CB(0, self.user_id)
        self.user_cb.hot_start()
        self.user_cf = User_CF(self.user_id)
        recommendations_cb = self.user_cb.run_model()
        recommendations_cf = self.user_cf.run_model_cf()
        self._load_games_db(recommendations_cb,recommendations_cf)
        self.display_recommended_games()

    def display_recommended_games(self):
        with self.output_area:
            clear_output()
            games_vbox = widgets.VBox()
            for game_id in self.get_recommended_games():
                game_name = df_games.loc[df_games['app_id'] == game_id]['app_name'].tolist()[0]
                game_label = widgets.Label(f'{game_id}: {game_name}')
                like_button = widgets.Button(description='Like', layout=widgets.Layout(width='auto'))
                view_button = widgets.Button(description='View', layout=widgets.Layout(width='auto'))

                if game_id in self.temp_liked_games:
                    like_button.description = 'Liked'
                    like_button.disabled = True
                else:
                    like_button.description = 'Like'
                    like_button.disabled = False

                like_button.on_click(lambda b, game_id=game_id: self.on_like_clicked(b, game_id))
                view_button.on_click(lambda b, game_id=game_id: self.display_game_details(b, game_id))
                buttons_hbox = widgets.HBox([like_button, view_button], layout=widgets.Layout(justify_content='flex-end'))
                game_hbox = widgets.HBox([game_label, buttons_hbox], layout=widgets.Layout(justify_content='space-between'))
                games_vbox.children += (game_hbox,)

            navigation_box = widgets.HBox([self.previous_page_button, self.refresh_button, self.next_page_button], layout=widgets.Layout(justify_content='space-between'))
            display(widgets.VBox([games_vbox, navigation_box]))


    def get_recommended_games(self):
        start = self.current_page * 10
        end = start + 10
        return self.recommended_games_ids[start:end]

    def on_next_page_clicked(self, b):
        self.current_page += 1
        self.display_recommended_games()

    def on_previous_page_clicked(self, b):
        if self.current_page > 0:
            self.current_page -= 1
        self.display_recommended_games()

    def on_back_clicked(self, b):
        self.display_recommended_games()

    def reset_current_page(self):
        self.current_page = 0

    def on_search_clicked(self, b):
        pass

    def display_game_details(self, b, game_id):
        with self.output_area:
            clear_output()
            game_tags_str = df_tags[df_tags['app_id'] == game_id]['tags'].iloc[0]
            game_tags = game_tags_str.split(' ')
            game_desc = df_desc[df_desc['id'] == game_id]['short_description'].iloc[0]
            tags_hbox = widgets.HBox(layout=widgets.Layout(flex_flow='row wrap', align_items='flex-start'))
            for tag in game_tags:
                tag = tag.replace('_', ' ').strip()
                tag_button = widgets.Button(description=tag, disabled=True,
                                            button_style='',
                                            layout=widgets.Layout(width='auto', margin='2px'))
                tags_hbox.children += (tag_button,)
            desc_html = widgets.HTML(
                value=f"<strong>Description:</strong> {game_desc}"
            )

            back_button = widgets.Button(description='Back')
            back_button.on_click(lambda b: self.on_back_clicked(b))
            display(widgets.VBox([tags_hbox, desc_html, back_button]))


    def on_like_clicked(self, b, game_id):
        self.temp_liked_games.append(game_id)
        b.description = 'Liked'
        b.disabled = True

    def on_refresh_clicked(self, b):
        for game_id in self.temp_liked_games:
            self.user_cb.like_game(game_id)
            self.user_cf.comment(game_id)
        self.temp_liked_games = []
        recommendations_cb = self.user_cb.run_model()
        recommendations_cf = self.user_cf.run_model_cf()
        self._load_games_db(recommendations_cb,recommendations_cf)
        self.reset_current_page()
        self.display_recommended_games()

# Usage
ui = SteamGameRecommenderUI()
ui.display()
76561198030410472


Output()

76561198119772292